# Training


### Building and compiling a model

In [1]:
%%capture
import h5py
from chimeranet.models import ChimeraPPModel

# probe shape of dataset and set embedding dimension
dataset_path = 'example-dataset-train.hdf5'
with h5py.File(dataset_path, 'r') as f:
    _, T, F, C = f['y/embedding'].shape
D = 20
cm = ChimeraPPModel(T, F, C, D)

# build_model returns Keras' Model object
model = cm.build_model()
model.compile(
    'rmsprop',
    loss={
        'embedding': cm.loss_deepclustering(),
        'mask': cm.loss_mask()
    },
    loss_weights={
        'embedding': 0.9,
        'mask': 0.1
    }
)

### Training a model

In [2]:
# load train and validation data
dataset_validation_path = 'example-dataset-validation.hdf5'
with h5py.File(dataset_path, 'r') as f:
    x_train = f['x'][()]
    y_train = {'mask': f['y/mask'][()], 'embedding': f['y/embedding'][()]}
with h5py.File(dataset_validation_path, 'r') as f:
    x_validation = f['x'][()]
    y_validation = {'mask': f['y/mask'][()], 'embedding': f['y/embedding'][()]}

# train model by model.fit function
model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_validation, y_validation),
    batch_size=32,
    epochs=10
)
# save the model
model_path = 'example-model.hdf5'
model.save(model_path)

Train on 160 samples, validate on 160 samples
Epoch 1/10
160/160 [==============================] - 261s 2s/step - loss: 18381.2736 - embedding_loss: 18457.8914 - mask_loss: 17691.7186 - val_loss: 12940.3148 - val_embedding_loss: 12466.7975 - val_mask_loss: 17201.9725
Epoch 2/10
160/160 [==============================] - 258s 2s/step - loss: 12468.0131 - embedding_loss: 12140.8994 - mask_loss: 15412.0373 - val_loss: 11921.1551 - val_embedding_loss: 11286.8416 - val_mask_loss: 17629.9813
Epoch 3/10
160/160 [==============================] - 266s 2s/step - loss: 12008.5932 - embedding_loss: 11784.1303 - mask_loss: 14028.7604 - val_loss: 11863.6486 - val_embedding_loss: 11170.0996 - val_mask_loss: 18105.5883
Epoch 4/10
160/160 [==============================] - 266s 2s/step - loss: 12022.4951 - embedding_loss: 11823.9797 - mask_loss: 13809.1371 - val_loss: 11875.8588 - val_embedding_loss: 11263.6596 - val_mask_loss: 17385.6521
Epoch 5/10
160/160 [==============================] - 266s 2s/